# gurobipy-pandas

The missing link between `pandas` and `gurobipy`!

In this session:

- Overview of `gurobipy-pandas` design
- Basic usage mechanics
- Complete modelling examples

# Design Principles

Optimization models define data, variables, and constraints over indexes:


$$
\begin{alignat}{2}
\max \quad        & \sum_{i \in I} \sum_{j \in J} p_{i} x_{ij} \\
\mbox{s.t.} \quad & \sum_{i \in I} w_{i} x_{ij} \le c_{j} & \forall j \in J \\
                  & \sum_{j \in J} x_{ik} \le 1 & \forall i \in I \\\
                  & x_{ij} \in \lbrace 0, 1 \rbrace & \forall i \in I, j \in J \\
\end{alignat}
$$

In [1]:
import pandas as pd
import numpy as np
product_data = pd.DataFrame(
    index=[1, 2, 3, 4, 5],
    columns=["cost_per_unit", "amount_available"],
    data=np.random.random((5, 2)).round(2)
)

Pandas DataFrames and Series define data over indexes:

In [2]:
product_data

,cost_per_unit,amount_available
1,0.80,0.11
2,0.33,0.97
3,0.27,0.66
4,0.66,0.35
5,0.13,0.29


## Missing Link?

Defining gurobipy variables and constraints over pandas indexes.

`gurobipy-pandas` provides simple functions to help here.

# Installation

```
pip install gurobipy-pandas
```

Then, add one more import to your arsenal:

In [3]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

import gurobipy_pandas as gppd

# Handy trick for live coding
gppd.set_interactive()

# Silence please
gp.setParam('OutputFlag', 0)

# Usage

- `gurobipy` objects are still the entry point for:
    - creating models
    - starting optimization
    - constants, status codes, etc
- `gurobipy_pandas` provides accessors and functions to:
    - create sets of variables based on indexes
    - create constraints based on aligned series
    - extract solutions as series

# Creating Models

The usual way

In [4]:
model = gp.Model()

# Creating Variables

- Using free functions

In [5]:
data = pd.DataFrame(
    {
        "i": [0, 0, 1, 2, 2],
        "j": [1, 2, 0, 0, 1],
        "u": [0.3, 1.2, 0.7, 0.9, 1.2],
        "c": [1.3, 1.7, 1.4, 1.1, 0.9],
    }
).set_index(["i", "j"])

In [6]:
data

u    c
i j          
0 1  0.3  1.3
  2  1.2  1.7
1 0  0.7  1.4
2 0  0.9  1.1
  1  1.2  0.9

In [7]:
x = gppd.add_vars(model, data, name="x", ub="u")
x

i  j
0  1    <gurobi.Var x[0,1]>
   2    <gurobi.Var x[0,2]>
1  0    <gurobi.Var x[1,0]>
2  0    <gurobi.Var x[2,0]>
   1    <gurobi.Var x[2,1]>
Name: x, dtype: object

# Creating Variables

- Using dataframe accessors

In [8]:
data

u    c
i j          
0 1  0.3  1.3
  2  1.2  1.7
1 0  0.7  1.4
2 0  0.9  1.1
  1  1.2  0.9

In [9]:
variables = (
    data
    .gppd.add_vars(model, name="y", ub="u")
    .gppd.add_vars(model, name="z", obj="c")
)
variables

u    c                    y                    z
i j                                                    
0 1  0.3  1.3  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>
  2  1.2  1.7  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>
1 0  0.7  1.4  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>
2 0  0.9  1.1  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>
  1  1.2  0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>

# Creating Expressions

- Pandas largely handles this for us

In [10]:
x.groupby("i").sum()

i
0        x[0,1] + x[0,2]
1    <gurobi.Var x[1,0]>
2        x[2,0] + x[2,1]
Name: x, dtype: object

In [11]:
2 * variables["y"] + variables["z"]

i  j
0  1    2.0 y[0,1] + z[0,1]
   2    2.0 y[0,2] + z[0,2]
1  0    2.0 y[1,0] + z[1,0]
2  0    2.0 y[2,0] + z[2,0]
   1    2.0 y[2,1] + z[2,1]
dtype: object

# Creating Constraints

- Using free functions

In [12]:
gppd.add_constrs(  
    model,
    variables.groupby("j")["y"].sum(),
    GRB.LESS_EQUAL,
    variables.groupby("i")["y"].sum(),
    name="c1",
)

0    <gurobi.Constr c1[0]>
1    <gurobi.Constr c1[1]>
2    <gurobi.Constr c1[2]>
Name: c1, dtype: object

# Creating Constraints

- Using dataframe accessors

In [13]:
vars_and_constrs = variables.gppd.add_constrs(  
    model, "y + z <= 1", name="c1"
)
vars_and_constrs

u    c                    y                    z  \
i j                                                       
0 1  0.3  1.3  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>   
  2  1.2  1.7  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>   
1 0  0.7  1.4  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>   
2 0  0.9  1.1  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>   
  1  1.2  0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>   

                          c1  
i j                           
0 1  <gurobi.Constr c1[0,1]>  
  2  <gurobi.Constr c1[0,2]>  
1 0  <gurobi.Constr c1[1,0]>  
2 0  <gurobi.Constr c1[2,0]>  
  1  <gurobi.Constr c1[2,1]>

# Setting Objectives

Generally one objective (not an index-wise operation)

So we use `gurobipy` methods here

In [14]:
(x * data["c"]).sum()

<gurobi.LinExpr: 1.3 x[0,1] + 1.7 x[0,2] + 1.4 x[1,0] + 1.1 x[2,0] + 0.9 x[2,1]>

In [15]:
model.setObjective((x * data["c"]).sum(), sense=GRB.MAXIMIZE)

# Extracting Solutions

(Solve the model first)

In [16]:
model.optimize()
x.gppd.X  # Series accessor

i  j
0  1    0.3
   2    1.2
1  0    0.7
2  0    0.9
   1    1.2
Name: x, dtype: float64

# Other Series accessor functionality

In [17]:
vars_and_constrs['c1']

i  j
0  1    <gurobi.Constr c1[0,1]>
   2    <gurobi.Constr c1[0,2]>
1  0    <gurobi.Constr c1[1,0]>
2  0    <gurobi.Constr c1[2,0]>
   1    <gurobi.Constr c1[2,1]>
Name: c1, dtype: object

In [18]:
vars_and_constrs['c1'].gppd.Slack

i  j
0  1    1.0
   2    1.0
1  0    1.0
2  0    1.0
   1    1.0
Name: c1, dtype: float64

In [19]:
expr = 2.0 * x + variables['y'] + 0.5
expr

i  j
0  1    0.5 + 2.0 x[0,1] + y[0,1]
   2    0.5 + 2.0 x[0,2] + y[0,2]
1  0    0.5 + 2.0 x[1,0] + y[1,0]
2  0    0.5 + 2.0 x[2,0] + y[2,0]
   1    0.5 + 2.0 x[2,1] + y[2,1]
dtype: object

In [20]:
expr.gppd.get_value()

i  j
0  1    1.1
   2    2.9
1  0    1.9
2  0    2.3
   1    2.9
dtype: float64

# Example

[Project-Team Allocation](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples/projects.html). Key points:

- *Before* taking any modelling steps: prepare your data properly
    - Clearly define your model indexes, and align your dataframes to these indexes
    - Keep data reading & cleaning separate from model building
- Solutions are read back as numeric pandas data
    - Plug directly into the rest of pydata ecosystem for post-processing
    - Work pandonically with the results

- Show mathematical model
- Talk through the indexes and data

- Show the prepared dataframes
- Correct dtypes, no missing values, sparse index

- Walk through the code

# More examples are available

https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples.html

# Performance

[Dedicated page in the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/performance.html). Key points:

- `gurobipy-pandas` won't magically make your model building code fast
- The API provides some structure to take *well-organized and prepared data* and help you build *clearly defined models* in the pandas style

# Final thoughts

- [Read the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/index.html)
    - Check the examples for clean patterns to emulate
- [Check the repo](https://github.com/Gurobi/gurobipy-pandas)
    - Bugs? Feature request? Open an issue
- [Discuss!](https://support.gurobi.com/hc/en-us/community/topics/10373864542609-GitHub-Projects%3E)
    - For usage questions

# Thanks!

Questions?